##При запуске других блоков, нужно запустить вот этот блок, чтобы скачать дата сет и определить переменную **df**

In [ ]:
import kagglehub
import pandas as pd
import os
import shutil

path = kagglehub.dataset_download('computingvictor/transactions-fraud-datasets')

source_path = f'{path}/transactions_data.csv'
destination_path = 'data/transactions_data.csv'

os.makedirs('data', exist_ok=True)
shutil.copy(source_path, destination_path)

df = pd.read_csv(destination_path)
print('Готово')

## Предобработка данных


### 1. Знакомство с данными
- Выведем первые и последние строки датасета.
- Случайную выборку строк.
- Атрибуты датафрейма: типы данных, количество ненулевых значений, использование памяти.



In [ ]:
print("=== Первые 5 строк ===")
display(df.head())

print("=== Последние 5 строк ===")
display(df.tail())

print("=== Случайная выборка строк ===")
display(df.sample(5, random_state=42))

print("=== Информация о датафрейме ===")
df.info(memory_usage='deep')

### 2. Выявление проблем в данных
- При первичном осмотре я обратил внимание на аномалии в числовых признаках, несоответствия в типах данных, пропуски, некорректные или неинформативные значения.


In [ ]:
print("\n=== Основная статистика числовых признаков ===")
print(df.describe())

print("\n=== Уникальные значения в категориальных столбцах ===")
categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    print(f"\nУникальные значения в {col}:")
    print(df[col].value_counts())

### 3. Корректность наименований столбцов
- Проверил имена столбцов на соответствие «змеиному регистру» (snake_case).
- Приведёл имена и текстовые значения к нижнему регистру.


In [ ]:
df.columns = [col.strip().lower() for col in df.columns]
print("\n=== Новые имена столбцов ===")
print(df.columns.tolist())

for col in categorical_cols:
    df[col] = df[col].str.lower()

### 4. Проверка и обработка пропусков
- Проверил наличие пропусков с помощью методов `isnull().sum()`.
- После проверки пропусков выявил достаточно много данных. Заполнил их нейтральными данными, чтобы все было корректно.
- Удалил не нужные столбцы `id, client_id, card_id`

In [ ]:
print("\n=== Пропуски в данных ===")
missing = df.isnull().sum()
print(missing[missing > 0])

df['merchant_state'] = df['merchant_state'].fillna('unknown')
print("Пропуски в merchant_state заполнены значением 'unknown'.")

df['zip'] = df['zip'].fillna(0)
print("Пропуски в zip заполнены значением 0 (нет данных).")

df['errors'] = df['errors'].fillna('no_error')
print("Пропуски в errors заполнены значением 'no_error'.")

print("Удаление не нужных столбцов id, client_id, card_id")
df.drop(columns=['id', 'client_id', 'card_id'], inplace=True)

print("\n=== Пропуски в данных ===")
missing = df.isnull().sum()
print(missing[missing > 0])

### 5. Приведение типов данных
#### Преобразовали столбцы:
- date -> datetime
- amount -> float64
- merchant_city -> string
- merchant_state -> string
- use_chip -> string
- errors -> string


In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

df['amount'] = df['amount'].replace('[$]', '', regex=True)
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

df['merchant_city'] = df['merchant_city'].astype('string')

df['merchant_state'] = df['merchant_state'].astype('string')

df['use_chip'] = df['use_chip'].astype('string')

df['errors'] = df['errors'].astype('string')

print("=== Информация о датафрейме ===")
df.info(memory_usage='deep')

### 6. Поиск и удаление дубликатов
- Проверим датафрейм на наличие дублирующихся строк и удалим их, если они есть.
- После проводим пересчет индексов


In [ ]:
duplicates = df.duplicated().sum()
print(f"\nКоличество дубликатов: {duplicates}")
if duplicates > 0:
    df.drop_duplicates(inplace=True)
    print(f"Удалены {duplicates} дубликатов.")

print("Пересчет индексов")
df.reset_index(drop=True, inplace=True)

### 7. Промежуточные выводы

- На этапе предобработки мы заполнили пустые данные, стандартизировали имена и значения, привели типы признаков к корректным форматам, проверили дубликаты.

In [ ]:
print(f"Финальный размер датасета: {df.shape}")
print("Предобработка завершена: пропуски обработаны, дубликаты удалены, имена столбцов приведены к единому стилю.")


## Исследовательский анализ данных (EDA)

В данном разделе проводится первичный визуальный и табличный анализ данных с целью выявления закономерностей, аномалий и зависимостей, способствующих решению бизнес-задачи — обнаружению подозрительных или мошеннических транзакций.


### 1. Индексация и логическая фильтрация

Здесь выполняется выборка данных с помощью позиционной и логической индексации. Это позволяет локально просматривать и анализировать конкретные участки датасета, выявлять нетипичные случаи и ориентироваться в структуре информации.


In [ ]:
print("1. Первые 5 строк с колонками от 1 до 4:")
display(df.iloc[:5, 1:5])

print("2. Последние 5 строк с определёнными столбцами:")
display(df.loc[df.index[-5:], ['amount', 'date']])

print("3. 10 случайных строк только с колонками amount и errors:")
display(df.loc[df.sample(10).index, ['amount', 'errors']])

print("4. Строки с чётными индексами:")
display(df.iloc[::2].head())

print("5. Строки по диапазону с шагом:")
display(df.iloc[1000:1100:10])

print("\n ================== \n")

print("1. Транзакции более 500:")
display(df[df['amount'] > 500])

print("2. Транзакции с ошибками:")
display(df[df['errors'] != 'no_error'])

print("3. Транзакции Chip Transaction:")
display(df[df['use_chip'] == 'chip transaction'])

print("4. Транзакции в штате CA:")
display(df[df['merchant_state'] == 'ca'])

print("5. Транзакции с MCC в категории 5411 (продуктовые магазины):")
display(df[df['mcc'] == 5411])

### 2. Сортировка данных

Анализ самых больших и самых маленьких значений по суммам транзакций и наличию ошибок помогает быстро выявить аномалии. Высокие суммы с ошибками и онлайн транзакц (Неверный пин или cvv) — потенциальные признаки мошенничества.


In [ ]:
print("Наибольшие суммы:")
display(df.sort_values(by='amount', ascending=False).head())

print("Наименьшие суммы:")
display(df.sort_values(by='amount', ascending=True).head())

print("Топ-10 транзакций с ошибками:")
display(df[df['errors'] != 'no_error'].sort_values(by='amount', ascending=False).head(10))


### 3. Фильтрация данных

Фильтрация через `query()` и `numpy.where()` позволяет отобрать только те строки, которые соответствуют условиям бизнес-логики (например, транзакции Online, с ошибками, из конкретного штата и т.п.). Это важно для фокусировки на подозрительных операциях.


In [ ]:
print("Где amount > 1000 и use_chip == 'online transaction':")
display(df.query("amount > 1000 and use_chip == 'online transaction'"))

print("Где merchant_state == 'tx' и errors != 'no_error':")
display(df.query("merchant_state == 'tx' and errors != 'no_error'"))

print("Где mcc == 5411 и amount < 100:")
display(df.query("mcc == 5411 and amount < 100"))

print("Где use_chip == 'chip transaction' и amount > 2000:")
display(df.query("use_chip == 'chip transaction' and amount > 2000"))

print("Где merchant_city == 'los angeles':")
display(df.query("merchant_city == 'los angeles'"))


import numpy as np

large_amount = np.where(df['amount'] > 1000)
print("Где amount > 1000:")
display(df.iloc[large_amount])

with_errors = np.where(df['errors'] != 'no_error')
print("Где errors != 'no_error':")
display(df.iloc[with_errors])

in_state = np.where(df['merchant_state'] == 'ny')
print("Где merchant_state == 'ny':")
display(df.iloc[in_state])

low_amount = np.where((df['amount'] < 100) & (df['use_chip'] == 'online transaction'))
print("Где amount < 100 и use_chip == 'online transaction':")
display(df.iloc[low_amount])

in_city = np.where(df['merchant_city'] == 'chicago')
print("Где merchant_city == 'chicago':")
display(df.iloc[in_city])


### 4. Сводные таблицы

Сводные таблицы позволяют агрегировать и структурировать данные по различным признакам. Например, можно увидеть, в каких штатах больше всего операций, какие категории MCC наиболее активны, и где чаще всего происходят ошибки.


In [ ]:
pivot1 = df.pivot_table(index='merchant_state', values='amount', aggfunc='count')
display(pivot1.sort_values(by='amount', ascending=False).head())

pivot2 = df.pivot_table(index='mcc', values='amount', aggfunc='mean')
display(pivot2.sort_values(by='amount', ascending=False).head())

pivot3 = df.pivot_table(index='merchant_state', columns='errors', values='date', aggfunc='count', fill_value=0)
display(pivot3.head())


### 5. Группировка и агрегирование

Использование groupby и агрегатных функций позволяет получить сводную статистику по ключевым признакам: сумма, среднее значение, количество операций. Это полезно для изучения общих трендов и понимания поведения пользователей в разных сегментах.


In [ ]:
agg1 = df.groupby('merchant_state')['amount'].agg(['sum', 'mean', 'max'])
display(agg1.sort_values(by='sum', ascending=False).head())

agg2 = df.groupby('mcc')['amount'].agg(['count', 'mean']).sort_values(by='count', ascending=False)
display(agg2.head())


### 6. Свободный анализ и наблюдения

На этом этапе анализируются интересные закономерности, которые могут быть полезны для построения модели. Например, частота использования чипа -> online transaction, наиболее распространённые ошибки и география операций с ошибками.


In [ ]:
print("\nЧастота ошибок:")
display(df['errors'].value_counts().head(10))

top_error_states = df[df['errors'] != 'no_error']['merchant_state'].value_counts().head()
print("\nГде больше всего ошибок:")
display(top_error_states)

chip_usage = df['use_chip'].value_counts(normalize=True)
print("\nИспользование чипа:")
display(chip_usage)


### 7. Промежуточные выводы по результатам исследовательского анализа данных

- С помощью индексации и логических фильтров удалось выделить группы транзакций, которые потенциально могут быть связаны с мошенничеством — например, операции чипа online transaction, транзакции с техническими ошибками и крупные суммы.
- Сортировка по различным признакам показала наличие аномально высоких сумм, в том числе среди транзакций с ошибками. Это указывает на возможные попытки обмана или технические сбои при обработке крупных операций.
- Фильтрация с использованием методов `query` и `where` позволила гибко исследовать подмножества данных: транзакции из определённых штатов, с конкретными MCC-кодами, и выявить закономерности между типом ошибки и характеристиками операций.
- Построенные сводные таблицы помогли обобщить данные: например, выяснилось, что определённые ошибки чаще возникают в конкретных регионах, а средние суммы транзакций значительно варьируются по категориям MCC.
- С помощью группировки и агрегирования (`groupby + agg`) были рассчитаны суммарные и средние значения по категориям — это позволило выявить наиболее активные торговые категории и географические зоны.
- Свободный анализ показал, что некоторые ошибки (например, `bad pin` и `insufficient balance`) повторяются особенно часто, а использование чипа online transaction распределено неравномерно — что может быть полезным при построении модели обнаружения мошенничества.

📌 В целом, анализ позволил выделить группы транзакций, которые отличаются по характеристикам от «нормального поведения» и заслуживают внимания при последующем построении модели или разработке бизнес-правил для антифрод-фильтрации.


## Статистический анализ данных

В этом разделе проводится анализ статистических характеристик данных, включая средние значения, корреляции, распределения и зависимости категориальных признаков.


### 1. Основные статистические показатели

- Признак `amount` (сумма транзакции) показывает положительную асимметрию: медиана — 28.99, среднее — 42.98, что указывает на наличие выбросов и длинного правого хвоста.
- Мода для `amount` — 80.0, что может свидетельствовать о частом использовании фиксированных сумм.
- `zip` имеет большое количество значений, равных 0 (мода = 0), что может быть техническим значением или заполнением пропусков.
- `mcc` — категориальный числовой признак — распределён относительно равномерно и стабильно.

In [ ]:
numeric_cols = df.select_dtypes(include='number').columns
print("=== describe() по числовым признакам ===")
display(df[numeric_cols].describe())

print("\n=== Медианы ===")
display(df[numeric_cols].median())

print("\n=== Мода ===")
display(df[numeric_cols].mode().iloc[0])

print("\n=== Стандартное отклонение ===")
display(df[numeric_cols].std())

print("\n=== 25%, 50%, 75% квартиль ===")
display(df[numeric_cols].quantile([0.25, 0.5, 0.75]))


### 2. Матрица корреляций

- Между числовыми признаками (`amount`, `merchant_id`, `zip`, `mcc`) не выявлено сильных взаимосвязей.
- Значения корреляции не превышают порог ±0.1.

📌 *Вывод:* признаки вносят независимую информацию, и могут быть полезны модели без риска мультиколлинеарности.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr = df[numeric_cols].corr()

plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Матрица корреляций между числовыми признаками')
plt.show()

high_corr = corr[(corr.abs() > 0.7) & (corr.abs() < 1.0)]
print("=== Высокие корреляции (|r| > 0.7) ===")
print(high_corr.dropna(how='all').dropna(axis=1, how='all'))


### 3. Распределения числовых переменных

- Распределение `amount` сильно смещено вправо, характерно наличие аномально высоких значений.
- Распределение `zip` имеет множество пиков и резко выраженный левый пик (в районе 0).


In [ ]:
for col in ['amount', 'zip']:
    if col in df.columns:
        plt.figure(figsize=(8, 4))
        sns.histplot(df[col], kde=True, bins=50)
        plt.title(f'Распределение признака {col}')
        plt.xlabel(col)
        plt.ylabel('Частота')
        plt.show()


### 4. Таблицы сопряжённости

- Ошибки (`errors`) чаще встречаются при транзакциях без чипа (`online`, `swipe`), тогда как `chip transaction` показывает >98% успеха.
- Частота ошибок типа `Insufficient Balance`, `Bad CVV`, `Bad PIN` заметно выше при отсутствии чипа.

📌 *Вывод:* тип транзакции (`use_chip`) — один из ключевых факторов при определении риска ошибки.

In [ ]:
cross_tab = pd.crosstab(df['use_chip'], df['errors'], normalize='index') * 100
print("=== Таблица сопряжённости use_chip и errors (в процентах) ===")
display(cross_tab)


### 5. Проверка гипотез (χ²-тест)

- Получено значимое значение χ²-статистики (158730.35) и **p-value = 0.0000**.
- Это означает, что существует статистически значимая зависимость между типом транзакции (`use_chip`) и возникновением ошибки (`errors`).

📌 *Вывод:* тип оплаты влияет на вероятность ошибки — это не случайная связь.

In [ ]:
from scipy.stats import chi2_contingency

contingency = pd.crosstab(df['use_chip'], df['errors'])
chi2, p, dof, expected = chi2_contingency(contingency)

print(f"Chi²-статистика: {chi2:.2f}")
print(f"p-value: {p:.4f}")
if p < 0.05:
    print("→ Есть статистически значимая зависимость между использованием чипа и наличием ошибки.")
else:
    print("→ Нет статистически значимой зависимости.")


### 6. Промежуточные выводы

- Статистический анализ показал, что признак `amount` (сумма транзакции) имеет положительную асимметрию и выраженные выбросы — это следует учитывать при масштабировании и отборе модели.
- Значение `zip` часто принимает значение 0, что может указывать на технические пропуски. Его стоит либо исключить, либо дополнительно обработать.
- Корреляционный анализ не выявил сильной связи (|r| > 0.7) между числовыми признаками — это свидетельствует об их независимости, что полезно для последующего моделирования.
- Графики распределения продемонстрировали, что числовые признаки имеют неоднородные формы распределения, а некоторые — выраженные пики и выбросы.
- Таблицы сопряжённости показали, что ошибки значительно чаще встречаются при онлайн- и swipe-транзакциях, чем при операциях с использованием чипа.
- Проведённый χ²-тест подтвердил статистически значимую зависимость между типом оплаты (`use_chip`) и возникновением ошибок (`errors`), что делает этот признак критически важным для антифрод-анализа.

*Итог:* Статистический анализ позволил выявить ключевые особенности данных, обнаружить неочевидные связи и подтвердить значимость некоторых категориальных признаков. Эти результаты создают прочную основу для этапа построения и обучения модели машинного обучения.


## Графический анализ данных

### 1. Построение диаграмм

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(df['amount'], bins=100, color='skyblue', edgecolor='black')
plt.title('Распределение суммы транзакций (amount)')
plt.xlabel('Сумма')
plt.ylabel('Частота')
plt.grid(True)
plt.show()

print('\n')

plt.figure(figsize=(14, 7))
df['errors'].value_counts().head(10).plot(kind='bar', color='coral')
plt.title('10 самых частых ошибок в транзакциях')
plt.ylabel('Количество')
plt.xticks(rotation=45, fontsize=10)
plt.grid(axis='y')
plt.show()

print('\n')

plt.figure(figsize=(6, 6))
df['use_chip'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90, colors=['lightgreen', 'orange', 'lightblue'])
plt.title('Распределение способов оплаты')
plt.ylabel('')
plt.show()


### 2 Настройка диаграмм

In [ ]:
plt.figure(figsize=(8, 4))
sns.boxplot(x='use_chip', y='amount', data=df)
plt.title('Распределение суммы транзакций по типу оплаты')
plt.xlabel('Тип оплаты')
plt.ylabel('Сумма')
plt.grid(True)
plt.show()

print('\n')

plt.figure(figsize=(10, 5))
sns.countplot(data=df[df['errors'] != 'no_error'], x='use_chip', hue='errors', order=df['use_chip'].value_counts().index)
plt.title('Ошибки по типу оплаты')
plt.xticks(rotation=45)
plt.legend(title='Тип ошибки', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.grid(axis='y')
plt.show()


### 3. Портрет типичного потребителя

На основе анализа данных можно выделить следующие характеристики типичного потребителя:

- Совершает транзакции в основном с использованием **чипа** (swipe transaction).
- Типичная сумма транзакции составляет **от 20 до 80 единиц**.
- **Ошибки практически не возникают** — доля транзакций без ошибок превышает 98%.
- Наиболее частые категории операций по коду MCC — **5499, 5411 и 5812** (продуктовые, супермаркеты и рестораны).
- Географически потребители распределены по разным штатам и индексам, но часть данных (например, `zip = 0`) требует уточнения.


## Выводы и рекомендации

### 1. Итоги аналитической работы:
- Проведена комплексная обработка, очистка и анализ транзакционных данных с Kaggle.
- Выявлены статистические и визуальные закономерности, подтверждённые расчётами и графиками.
- Проведены проверка гипотез, кросс-табуляции и визуализации, описывающие связи между способом оплаты и ошибками.

### 2. Ответ на бизнес-задачу:
- Да, бизнес-задача решена: мы определили, какие транзакции чаще сопровождаются ошибками.
- Подтверждено, что **тип оплаты (использование чипа)** существенно влияет на успешность операции.

### 3. Практические рекомендации:
- Стимулировать клиентов использовать оплату с чипом — она заметно снижает вероятность ошибок.
- Реализовать мониторинг частых типов ошибок и аномалий (например, `bad pin`, `bad cvv`).
- Рассмотреть логарифмирование суммы при построении модели (из-за выбросов).

### 4. Возможные пути продолжения:
- Построение модели машинного обучения для автоматического выявления подозрительных транзакций.
- Реализация визуальной панели (дашборда) для мониторинга ключевых метрик.
- Проведение сегментации клиентов и выявление групп с повышенным риском.
